In [1]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
import joblib

In [2]:

train_21 = pd.read_csv('male_players (legacy).csv')

train_22 = pd.read_csv('players_22.csv')

C:\Users\ok\AppData\Local\Temp\ipykernel_13524\2037869050.py:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  train_21 = pd.read_csv('male_players (legacy).csv')
C:\Users\ok\AppData\Local\Temp\ipykernel_13524\2037869050.py:3: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  train_22 = pd.read_csv('players_22.csv')


Checking and dropping variables with more than 30% missing values

In [3]:

threshold = 0.3 * train_21.shape[0]
train_21 = train_21.dropna(thresh=threshold, axis=1)
train_21.drop(columns=['wage_eur','value_eur', 'international_reputation', 'release_clause_eur'], inplace=True)


Selecting Numeric Features

In [4]:
# selecting numerical features
numerical_features = train_21.select_dtypes(include =[np.number])

Multivariable Imputation on numerical variables

In [5]:

imp = SimpleImputer(strategy='mean')
numerical_features = pd.DataFrame(imp.fit_transform(numerical_features), columns=numerical_features.columns, index=numerical_features.index)

In [6]:
#Specifying dependent and independent variables
y = numerical_features['overall']
X = numerical_features.drop(columns = ['overall'])

Feature Importance

In [7]:

selector = SelectKBest(score_func=f_regression, k=14)  # Select top 12 features
X_new = selector.fit_transform(X, y)
important_features = X.columns[selector.get_support()]
#important_feature_indices = selector.get_support()
#important_features = X.columns[important_feature_indices]
train_21[important_features]

,potential,age,shooting,passing,dribbling,physic,attacking_short_passing,skill_curve,skill_long_passing,skill_ball_control,movement_reactions,power_shot_power,mentality_vision,mentality_composure
0,95,27,89.0,86.0,96.0,63.0,89,89,76,96,94,80,90,NaN
1,92,29,93.0,81.0,91.0,79.0,82,88,72,92,90,94,81,NaN
2,90,30,86.0,83.0,92.0,64.0,86,85,76,90,89,86,84,NaN
3,90,32,91.0,81.0,86.0,86.0,84,80,76,90,85,93,83,NaN
4,90,28,NaN,NaN,NaN,NaN,42,25,41,31,89,42,20,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,61,18,48.0,40.0,47.0,44.0,38,39,33,45,39,64,42,40.0
161579,58,19,25.0,29.0,34.0,57.0,30,24,25,32,42,33,31,35.0
161580,58,19,36.0,43.0,46.0,53.0,51,30,43,35,50,51,40,35.0
161581,70,17,50.0,36.0,46.0,42.0,40,38,26,43,45,42,44,43.0


In [8]:
headers = numerical_features.columns

In [9]:
X = pd.DataFrame(X_new, columns=important_features, index = X.index)

Splitting data to training and testing

In [10]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X_new,y,test_size=0.2,random_state=42,stratify=y)

In [11]:
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
import pickle as pkl

In [12]:
dtree = DecisionTreeRegressor(max_depth=15)
lregressor = LinearRegression()
sdg = SGDRegressor()
svr = LinearSVR(C=1.0)

### Training Models

In [ ]:
for model in (dtree, lregressor, sdg, svr):
    model.fit(Xtrain, Ytrain)

    y_pred = model.predict(Xtest)

    print(f'''\n {model.__class__.__name__}: \n Mean Absolute error: {mean_absolute_error(y_pred, Ytest)} \n Mean Squared Error: {mean_squared_error(y_pred, Ytest)} \n R2_score: {r2_score(y_pred, Ytest)}''')



 DecisionTreeRegressor: 
 Mean Absolute error: 1.012959729530046 
 Mean Squared Error: 2.4791336241674 
 R2_score: 0.9489494093962051

 LinearRegression: 
 Mean Absolute error: 1.6047062618474033 
 Mean Squared Error: 4.2852389757744165 
 R2_score: 0.9058330652657282

 SGDRegressor: 
 Mean Absolute error: 1076419048327.5916 
 Mean Squared Error: 1.3165161470170088e+24 
 R2_score: -5.348995035195485


In [ ]:
#Preparing testing dataset
train_22_cleaned = train_22.select_dtypes(include=[np.number]).dropna(axis=1)
common_features = list(set(X.columns) & set(train_22_cleaned.columns))
    
new_X = X[common_features]
x_text22 = train_22_cleaned[common_features]
    

In [ ]:
#Training models again
for model in (dtree, lregressor, sdg, svr):
    #Spliting into trainig and testing
    X_train,X_test,Y_train,Y_test=train_test_split(new_X,y,test_size=0.2,random_state=42,stratify=y)
    model.fit(X_train, Y_train)
    
    y_pred = model.predict(Xtest)

    print(f'''\n {model.__class__.__name__}: \n Mean Absolute error: {mean_absolute_error(y_pred, Y_test)} \n Mean Squared Error: {mean_squared_error(y_pred, Y_test)} \n R2_score: {r2_score(y_pred, Y_test)}''')


In [ ]:
#Evaluating models on testing dataset
for model in (dtree, lregressor, sdg, svr):
    X_train,X_test,Y_train,Y_test=train_test_split(new_X,y,test_size=0.2,random_state=42,stratify=y)
    model.fit(X_train, Y_train)
    
    y_pred = model.predict(X_test22)
    
    print(f'''\n {model.__class__.__name__}: \n Mean Absolute error: {mean_absolute_error(y_pred, Y_test)} \n Mean Squared Error: {mean_squared_error(y_pred, Y_test)} \n R2_score: {r2_score(y_pred, Y_test)}''')

    

## Ensemble Learning

### Grid Search with Cross Validation

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor

In [ ]:
cv=KFold(n_splits=3)
rf = RandomForestRegressor()
rf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf_grid_search = GridSearchCV(estimator=rf, param_grid=rf_param_grid, cv=cv, n_jobs=-1, verbose=2)

start_time = time.time()
rf_grid_search.fit(Xtrain, Ytrain)
rf_duration = time.time() - start_time

# Save the model
pkl.dump(rf_grid_search, open("best_model_predictor" + '.pkl', 'wb'))

y_pred = rf_grid_search.predict(X_test)


print(f"Random Forest Grid Search Duration: {rf_duration} seconds")

print(f'''\n {model.__class__.__name__}: \n Mean Absolute error: {mean_absolute_error(y_pred, Ytest)} \n Mean Squared Error: {mean_squared_error(y_pred, Ytest)} \n R2_score: {r2_score(y_pred, Ytest)}''')


In [ ]:
# Getting the best parameters and best score
best_params = rf_grid_search.best_params_
best_score = rf_grid_search.best_score_
best_params

In [ ]:
best_score